In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
text=open('input.txt','r',encoding='utf-8').read()

In [3]:
ch=sorted(set(text))
stoi={s:i for i,s in enumerate(ch)}
itos={i:s for s,i in stoi.items()}
vocab_size=len(ch)

In [4]:
len(text)

1115394

In [4]:
print(''.join(ch))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
encode= lambda s:torch.tensor(list(stoi[i] for i in s),dtype=torch.long)
decode=lambda i:(''.join(itos[s.item()] for s in i))

In [6]:
print(encode(' hi Anmol'))
print(decode(encode(' hi Anmol')))

tensor([ 1, 46, 47,  1, 13, 52, 51, 53, 50])
 hi Anmol


In [7]:
data=encode(text)
data.shape,data.dtype

(torch.Size([1115394]), torch.int64)

In [8]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [9]:
block_size=8
x=train_data[:block_size+1]
print(x)
print(decode(x))

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
First Cit


In [10]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for i in range(block_size):
    context=x[:i+1]
    target=y[i]
    print(f'context is {context} and target is {target}')
    print(f'context is {decode(context)} and target is {itos[target.item()]}\n')


context is tensor([18]) and target is 47
context is F and target is i

context is tensor([18, 47]) and target is 56
context is Fi and target is r

context is tensor([18, 47, 56]) and target is 57
context is Fir and target is s

context is tensor([18, 47, 56, 57]) and target is 58
context is Firs and target is t

context is tensor([18, 47, 56, 57, 58]) and target is 1
context is First and target is  

context is tensor([18, 47, 56, 57, 58,  1]) and target is 15
context is First  and target is C

context is tensor([18, 47, 56, 57, 58,  1, 15]) and target is 47
context is First C and target is i

context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) and target is 58
context is First Ci and target is t



In [11]:
train_data[1]

tensor(47)

In [12]:
torch.randint(9,(10,))

tensor([4, 5, 1, 5, 6, 3, 2, 2, 0, 5])

# torch.stack
if we have list of tensor then we cannot use that list as tensor so we have to convert it to tensor 

if we try to wrap that list in torch.tensor api then it fails because list contains tensor 

hence to convert it to tensor we use torch .stack


In [22]:
torch.manual_seed(1334)
block_size=8
batch_size=4

def get_batch(split):
    data=train_data if split =='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack([data[i:block_size+i] for i in ix])
    y=torch.stack([data[i+1:block_size+i+1] for i in ix])
    return x,y

xtrain_b,ytrain_b=get_batch('train')
print('\t\tcontext')
print(xtrain_b.shape)
print(xtrain_b)
print('\n')

print('\t\ttarget')
print(ytrain_b.shape)
print(ytrain_b)

xval_b,yval_b=get_batch('val')



		context
torch.Size([4, 8])
tensor([[58,  8,  0,  0, 24, 33, 15, 21],
        [30, 21, 15, 20, 13, 30, 16,  1],
        [ 0, 13, 57,  1, 51, 59, 41, 46],
        [51, 43,  8,  1, 28, 56, 53, 41]])


		target
torch.Size([4, 8])
tensor([[ 8,  0,  0, 24, 33, 15, 21, 27],
        [21, 15, 20, 13, 30, 16,  1, 21],
        [13, 57,  1, 51, 59, 41, 46,  1],
        [43,  8,  1, 28, 56, 53, 41, 43]])


In [14]:
for i in range(batch):
    for j in range(block_size):
        context=xtrain_b[i][:j+1]
        target=ytrain_b[i][j]
        print(f'context is {context.tolist()} and target is {target}')


context is [58] and target is 8
context is [58, 8] and target is 0
context is [58, 8, 0] and target is 0
context is [58, 8, 0, 0] and target is 24
context is [58, 8, 0, 0, 24] and target is 33
context is [58, 8, 0, 0, 24, 33] and target is 15
context is [58, 8, 0, 0, 24, 33, 15] and target is 21
context is [58, 8, 0, 0, 24, 33, 15, 21] and target is 27
context is [30] and target is 21
context is [30, 21] and target is 15
context is [30, 21, 15] and target is 20
context is [30, 21, 15, 20] and target is 13
context is [30, 21, 15, 20, 13] and target is 30
context is [30, 21, 15, 20, 13, 30] and target is 16
context is [30, 21, 15, 20, 13, 30, 16] and target is 1
context is [30, 21, 15, 20, 13, 30, 16, 1] and target is 21
context is [0] and target is 13
context is [0, 13] and target is 57
context is [0, 13, 57] and target is 1
context is [0, 13, 57, 1] and target is 51
context is [0, 13, 57, 1, 51] and target is 59
context is [0, 13, 57, 1, 51, 59] and target is 41
context is [0, 13, 57, 

In [26]:
import torch.nn as  nn
from torch.nn import functional as f

torch.manual_seed(1334)
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.embeddingtable=nn.Embedding(vocab_size,vocab_size)


    def forward(self,idx,target=None):
        
        logits=self.embeddingtable(idx)#B,T,C
        if target is None:
            loss=None
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            target=target.view(B*T)
            loss=F.cross_entropy(logits,target)
        
        return logits,loss

    def generate(self,idx,max_new_tokens):
        for i in range(max_new_tokens):
            logits,loss=self(idx)
            logits=logits[:,-1,:]
            prob=F.softmax(logits,dim=1)
            idx_next=torch.multinomial(prob,num_samples=1)
            idx=torch.cat((idx,idx_next),dim=1)
        return idx 

model=BigramLanguageModel(vocab_size)
out,loss=model(xtrain_b,ytrain_b)
print(out.shape)
print(loss)
print(decode(model.generate(torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0]))

torch.Size([32, 65])
tensor(4.5510, grad_fn=<NllLossBackward0>)

ZHyiDIj-bDB3b;M$ruByFpk
EQOHseLAbn,&LYEkftK.MD3Utd'l&XLRZCyrm?kDk.pKEF!Cs,dx3UG&LJPpuCazNr,
 p;?Vt.$


In [27]:
optimizer=torch.optim.AdamW(model.parameters(),lr=1e-3)

In [35]:
max_steps=10000
batch_size=32
for _ in range(max_steps):
    xb,yb=get_batch('train')
    logits,loss=model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.407564401626587


In [39]:
print(decode(model.generate(torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0]))


Yoll r ureend t bocowavell yoknseeat, g bes 'd LESou femomy, dey oveealotha aind wigssio hintrenvere MOnceethe;
Faldime d ICO:
HE:
SII I ssthidimee grupe t touk dessend t therisou shed Sorund hy Ox ho our ch PEYoou. agu! tlor

Y thousedieelle ththith wofth. Gour:
Fo st, s p
We as d that m bishey foug mionde ber.
TIZonrar.
BRLENGEO'e k glathe
Sot, bayoul I wisis BUS:
And my l all powep thouray bullesamy theimay lletot Yor, o alldstes, oma thawe uste it LAng siry heakiobin st thar?

Whe y.

I ingo


# self attention

In [9]:
torch.manual_seed(1337)
import torch
B,T,C=2,4,2
x=torch.randn(B,T,C)
xbow=torch.zeros(B,T,C)#xbow=torch.zeros_like(x)
for b in range(B):
    for t in range(T):
        xprev=x[b,:t+1]
        xbow[b,t]=xprev.mean(0,keepdim=True)

In [10]:
xbow[0],x[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238]]),
 tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643]]))

In [25]:
torch.manual_seed(45)
a=torch.tril(torch.ones(3,3))
a=a/a.sum(1,keepdim=True)
b=torch.randint(9,(3,2)).float()
c=a@b
print(f'a= {a}\n')
print(f'b= {b}\n')
print(f'c= {c}\n')

a= tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

b= tensor([[5., 5.],
        [7., 8.],
        [6., 1.]])

c= tensor([[5.0000, 5.0000],
        [6.0000, 6.5000],
        [6.0000, 4.6667]])



In [33]:
wei=torch.tril(torch.ones(T,T))
wei=wei/wei.sum(1,keepdim=True)
xbow2=wei@x  

In [34]:
torch.allclose(xbow,xbow2)

True

In [35]:
tril=torch.tril(torch.ones(T,T))
wei=torch.zeros(T,T)
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,1)
xbow3=wei @ x

In [37]:
torch.allclose(xbow2,xbow3)

True

In [41]:
tril=torch.tril(torch.ones(T,T))
tril

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])

In [44]:
wei=torch.zeros(T,T)
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])

In [51]:
B,T,C=4,8,32
x=torch.randn(B,T,C)
tril=torch.tril(torch.ones(T,T))
wei=torch.zeros(T,T)
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,1)
xbow3=wei @ x


In [52]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [53]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
n_head=16
B,T,C=4,8,32
query=nn.Linear(C,n_head,bias=False)
key=nn.Linear(C,n_head,bias=False)
value=nn.Linear(C,n_head,bias=False)
q=query(x) #(B,T,H)
k=key(x)   #(B,T,H)
wei= q @ k.transpose(-2,-1)

tril=torch.tril(torch.ones(T,T))
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,1)

v=value(x)
xbow=wei @ v
wei

In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F

a=torch.randn(2,4,2)
b=torch.randn(2,4,2)
c=torch.cat((a,b),-1)
print(a)
print(b)
print(c)

tensor([[[ 1.0356, -0.6821],
         [ 1.2644, -0.4379],
         [-2.4376,  0.8240],
         [ 0.8788, -0.8962]],

        [[ 1.2942,  0.1899],
         [ 0.5598,  2.5771],
         [ 0.4137,  0.6907],
         [-0.7154,  0.2576]]])
tensor([[[-0.9912,  1.2821],
         [ 1.3425, -0.2810],
         [-0.2566, -1.0907],
         [-1.0971,  1.0692]],

        [[ 0.4258,  0.5840],
         [-1.0741,  1.0399],
         [ 0.4238, -0.3983],
         [-0.8265, -0.3131]]])
tensor([[[ 1.0356, -0.6821, -0.9912,  1.2821],
         [ 1.2644, -0.4379,  1.3425, -0.2810],
         [-2.4376,  0.8240, -0.2566, -1.0907],
         [ 0.8788, -0.8962, -1.0971,  1.0692]],

        [[ 1.2942,  0.1899,  0.4258,  0.5840],
         [ 0.5598,  2.5771, -1.0741,  1.0399],
         [ 0.4137,  0.6907,  0.4238, -0.3983],
         [-0.7154,  0.2576, -0.8265, -0.3131]]])
